# Compute ESM2-3B Scores for NucB

#### Prepare environment and import modules

In [6]:
# Load packages
%run setup_environment.py
import seq2fitness_models as models
import datasets
import seq2fitness_traintools as traintools
import seq2fitness_train as train
import sequence_utils
import torch.nn as nn
from pprint import pprint

import torch
import esm

Paths added to sys.path:
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/data
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/representation
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/representation/prep
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/utils
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/end_to_end_models
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/training
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/mld
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/mld/libdesign
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/inference
/home/ubuntu/DEML-Virginia/NucB-training/DEML/src/models
/home/ubuntu/DEML-Virginia/NucB-training/DEML/EvoProtGrad/evo_prot_grad
/home/ubuntu/DEML-Virginia/NucB-training/DEML/EvoProtGrad/evo_prot_grad/experts


#### Get ESM2-3B masked marginal scores

In [7]:
seq = """MIKKWAVHLLFSALVLLGLSGGAAYSPQHAEGAARYDDVLYFPASRYPETGAHISDAIKAGHADVCTIERSGADKRRQES
LKGIPTKPGFDRDEWPMAMCEEGGKGASVRYVSSSDNRGAGSWVGNRLNGYADGTRILFIVQ""".replace('\n','')

In [30]:
# Prepare CSV of mutants
AMINOS = "ACDEFGHIKLMNPQRSTVWY"
mutants = []
for i,aa in enumerate(seq):
    for amino in AMINOS:
        mutant = f"{aa}{i+1}{amino}"
        mutants.append(mutant)
positions = [int(mut[1:-1]) for mut in mutants]
df = pd.DataFrame({'mutant': mutants, 'position': positions})
df.to_csv('esm2_scores.csv')


In [18]:
!git clone https://github.com/facebookresearch/esm.git

Cloning into 'esm'...
remote: Enumerating objects: 1511, done.
remote: Counting objects: 100% (725/725), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 1511 (delta 567), reused 531 (delta 531), pack-reused 786 (from 1)
Receiving objects: 100% (1511/1511), 12.87 MiB | 15.61 MiB/s, done.
Resolving deltas: 100% (952/952), done.
Updating files: 100% (476/476), done.


In [26]:
cd esm/examples/variant-prediction

/home/ubuntu/DEML-Virginia/NucB-training/DEML/projects/NucB/esm/examples/variant-prediction


In [31]:
!python predict.py \
    --model-location esm2_t36_3B_UR50D \
    --sequence "MIKKWAVHLLFSALVLLGLSGGAAYSPQHAEGAARYDDVLYFPASRYPETGAHISDAIKAGHADVCTIERSGADKRRQESLKGIPTKPGFDRDEWPMAMCEEGGKGASVRYVSSSDNRGAGSWVGNRLNGYADGTRILFIVQ"\
    --dms-input ../../../esm2_scores.csv \
    --mutation-col mutant \
    --dms-output ../../../esm2_scores.csv \
    --offset-idx 1 \
    --scoring-strategy masked-marginals

Transferred model to GPU
100%|█████████████████████████████████████████| 144/144 [00:10<00:00, 13.62it/s]


In [ ]:
cd ../../..

In [37]:
# Convert scores from 1D to 2D
df = pd.read_csv('esm2_scores.csv', index_col=0)
loc = df.columns[0] 
del df[loc] # Delete added index column
df

,mutant,position,esm2_t36_3B_UR50D
0,M1A,1,-10.406944
1,M1C,1,-12.032123
2,M1D,1,-11.411701
3,M1E,1,-11.455269
4,M1F,1,-11.333991
...,...,...,...
2835,Q142S,142,-0.931034
2836,Q142T,142,-1.907223
2837,Q142V,142,-2.645308
2838,Q142W,142,-2.890419


In [51]:
# Scores as 2D dataframe
scores = {}
for i in set(df['position']):
    dfsel = df[df['position']==i]
    scores[i] = dfsel.iloc[:,-1].values
df = pd.DataFrame(scores).transpose()
df.columns = list(AMINOS)
df.insert(0, 'WT', list(seq))

# Save
df.to_excel('../data/NucB_esm2_3B_scores.xlsx', sheet_name='matrix')
df